# Development Notebooks

## Create Points for StreamStats

Usage Notes:


Outputs:


by Shane Putnam: sputnam@dewberry.com & Seth Lawler: slawler@dewberry.com

In [1]:
import os
import pandas as pd
import geopandas as gpd
import collections 
from osgeo import gdal, ogr,osr
from shapely.geometry import Point
from StreamStats_Points import*

### Load the masked stream grid:

In [2]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was masked by the catchment polygon

In [3]:
sg = StreamGrid(in_tif) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) #Print the value

epsg: 5070


In [4]:
df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

In [5]:
lat=1925315.186 #latitude of the pourpoint at the catchment outlet
lon=1616784.964 #longitude of the pourpoint

In [6]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print(pourpoint)

[(877, 1848)]


### Move up the stream and identify the confluences:

In [7]:
def MoveUpstream(df, stream_cell, nogo):
    """This function searches the 8 cells surrounding it to determine the location of the next stream cell(s).
        Arguments: df=the dataframe containing the stream raster data, stream_cell=the current stream cell that will be used to search for the next stream cell,
        nogo=list of stream cells that do not want to return as a new stream cell
    """
    row=stream_cell[0] #Extract the raster row 
    col=stream_cell[1] #Extract the raster column
    
    cell_value= df[row][col] #Determine the value of the cell
    assert cell_value == 1 #The value of the cell must be equal to the value associated with a stream cell
    
    nogo.append((row,col))   

    stream_cell=[]
    
    for i in range(-1,2): #For -1, 0, 1 in the vertical direction (move up and down rows)
        for j in range(-1,2): #For -1, 0, 1 in the horizontal direction (move across columns)
            value = df[row + i][col+j] #Read value of raster cell
            if value == 0: # if value is zero, no stream in this cell
                continue #loop back
            elif value==1 and (row+i,col+j) not in nogo: # if value is 1 and the cell is not in the nogo list, then...
                stream_cell.append((row + i,col+j)) #Add the new cell or cells to the stream_cell list
    return stream_cell, nogo    

In [7]:
def FindConfluence(df, stream_cell, nogo): #Keep moving upstream until you find a confluence--problem: might hit a dead end, maybe add an else if it is the end?
    """Function which repeates the MoveUpstream function until it finds two or more stream cells surrounding the provided stream_cell, indicating a confluence
    """
    count=0
    while len(stream_cell)==1: #While there is only 1 stream cell returned as we move up stream, keep moving up stream
        stream_cell, nogo=MoveUpstream(df, stream_cell[0], nogo)
        count+=1
    if len(stream_cell)>1: #If the number of stream cells 
        nogo=list(set(nogo+stream_cell))
    return stream_cell, nogo, count

In [7]:
def NextConfluence(df, confl, cellnum, nogo):
    """
    """
    stream_cell, nogo=MoveUpstream(df, confl[cellnum[0]]['pts'][cellnum[1]], nogo) #Move up one cell from the confluence
    if len(stream_cell)==1:
        stream_cell, nogo=MoveUpstream(df, stream_cell[0], nogo) #And move up one more time 
        if len(stream_cell)==1:
            if 'confl' not in confl[cellnum[0]]: #Add the stream_cell that is located three up from the intial split point.
                confl[cellnum[0]]['confl']=stream_cell  
            else:
                confl[cellnum[0]]['confl']=confl[cellnum[0]]['confl']+stream_cell
            stream_cell, nogo, count=FindConfluence(df, stream_cell, nogo) #Conf1 is empty if end 
            if len(stream_cell)>1:
                confl[max(list(confl.keys()))+1]={'pts':stream_cell, 'npts':len(stream_cell)}
        elif len(stream_cell)>1:
            confl[cellnum[0]]['pts']=confl[cellnum[0]]['pts']+stream_cell
            confl[cellnum[0]]['npts']=len(confl[cellnum[0]]['pts'])
    elif len(stream_cell)>1:
        confl[cellnum[0]]['pts']=confl[cellnum[0]]['pts']+stream_cell
        confl[cellnum[0]]['npts']=len(confl[cellnum[0]]['pts'])
    return confl, nogo    

In [7]:
cnum=0 #The confluence ID, start it at zero
snum=0 #The confluence number at a specific ID, start it at zero
cellnum=(cnum,snum) #Combine the confluence ID and number into a signle touple

confl=collections.OrderedDict() #Create an empty oredered dictionary to store the confluence information
confl[cnum]=confl[cnum]={"pts":pourpoint,"npts":len(pourpoint)} #Add the pourpoint as a confluence to this ditionary, specifying the pourpoint as "pts" and the number of points as "npts"

nogo=[] #Empty list to store the nogo points, i.e. the stream cells we previously searched

confl3=[] #Empty list to store the confluence locations that are three cells from the original junction. 

In [8]:
l=len(confl.keys())-1 
j=0
while j<=l: 
    k=confl[j]['npts']-1 
    i=0 
    while i<=k: 
        confl, nogo=NextConfluence(df, confl, (j,i), nogo)
        k=confl[j]['npts']-1 
        i+=1 
    l=len(confl.keys())-1 
    j+=1 

In [10]:
confl

{'pts': [(877, 1848)], 'npts': 1, 'confl': [(878, 1846)]}

In [154]:
confl3=[] #Empty list to store the row and column of each confluence point that is three cells away from the original junction
for i in range(len(confl.keys())):
    if 'confl' in confl[i]:    
        if len(confl[i]['confl'])>1:
            confl3=confl3+confl[i]['confl']

In [11]:
ppoints=[] #Empty list to store the row and column of each confluence
for i in range(len(confl.keys())):
    ppoints=ppoints+confl[i]['pts']

### Save the conflunces as a shapefile:

In [12]:
longitude, latitude=index2coord(sg, ppoints)

In [13]:
gdf=geodataframe(longitude, latitude, crs)

gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\allconfluences.shp')

gdf.head(2)

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


,Lon,Lat,Coordinates
0,1616785.0,1925315.0,POINT (1616785 1925315)
1,1616745.0,1925775.0,POINT (1616745 1925775)
